In [638]:
import binascii
import pprint
import hashlib
import secrets

def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'

def chunk(l, n):
    return [l[i:i + n] for i in range(0, len(l), n)]

import random

def flip_str(s):
    new_str = ''
    for i in range(len(s)):
        if s[i] == '0':
            new_str += '1'
        else:
            new_str += '0'
    return new_str

def generate_mixture_and_proof(c):
    c = chunk(text_to_bits(c), 4)
    
    bool_proof = ''
    mixture = []
    for cc in c:
        mix_c = cc
        if random.random() < 0.9:
            mix_c = flip_str(mix_c)
            bool_proof += '1'
        else:
            bool_proof += '0'
        mixture.append(mix_c)
    proof = hex(int(bool_proof, 2))
    proof = hashlib.sha3_256(bytearray.fromhex(proof[2:])).hexdigest()
    return hex(int(''.join(mixture), 2)), proof

def solve_puzzle(puzzle, message):
    bin_p = bin_from_hexstring(puzzle)
    bin_m = text_to_bits(message)
    
    while len(bin_p) < len(bin_m):
        bin_p = '0' + bin_p
    
    chunk_p = chunk(bin_p, 4)
    chunk_m = chunk(bin_m, 4)
    
    comp = list(zip(chunk_p, chunk_m))
    bool_proof = ''
    for c in comp:
        if c[0] == c[1]:
            bool_proof += '0'
        else:
            bool_proof += '1'
    
    proof = hex(int(bool_proof, 2))
    proof = hashlib.sha3_256(bytearray.fromhex(proof[2:])).hexdigest()
    return(proof)

def bin_from_hexstring(h):
    return bin(int(h[2:], 16))[2:]

In [660]:
# generate arbitrary data
block = secrets.token_urlsafe(128)
print(block)
print('\n')
# create a puzzle, which is a fuzzed out version of the secret, and a proof, which is a hash of the binary data that
# chunks each 4 bits out and marks if the original data differs from the puzzle or not. This allows for 90%+ of the
# data to be obfuscated and passed to someone else and a correct hash to be returned in very little time

puzzle, proof = generate_mixture_and_proof(block)
print(puzzle)
print('\n')
print(proof)

oWSQAMyE7gyQ3Kxn8yYC4_LmCAY3oTMU56IdMEP-_WWZ25kQMUjbvf6x7wo2BHqahbGelSVq4t2OQDqm5WGndqkIvObL3KHyO2_TMqFPvVDtSgRixXSHoq7u6bktzpCLrsAIUe5HgcZyHeEgN72sUm7A_CQYMvl7IcYynMkaOMg


0x90a8aca1beb289b5c89876aeccb47791c786a6bccba0b392bcb1a6cc90abb2aacac9b69bb2baafd2a0a858a5cdca94aeb25a959d8999c987c88890c2bdb78e9e979db89a93aca98ecb7bcd4faebb8e92caa848919b8e9b4686b09db3ccb4b776b0cda0abb28eb9af79a6bb8bac98ad9687a7acb7908ec885399d648b858fbcb38d8cbeb65595cab76893a586b79aba98b1c8cd8caa92c8bea0bcaea6bd8993c8b69ca68691b29491b0b298


4d24f53fdec332b7cc0dd27ceab86c62caf574b7cce77d2da4c4cd2b3be78814


In [661]:
# to solve the puzzle, the other party (which has the secret but just needs to prove that they do) looks over the
# puzzle and picks out which of the bits have been flipped. These are appended to a chunk of data where flipped = 1,
# and not flipped = 0. This bitstring is then turned into a hex representation and hashed. Thus, provability 
proof2 = solve_puzzle(puzzle, block)

print(proof2)

4d24f53fdec332b7cc0dd27ceab86c62caf574b7cce77d2da4c4cd2b3be78814


In [662]:
print(proof)

4d24f53fdec332b7cc0dd27ceab86c62caf574b7cce77d2da4c4cd2b3be78814


In [645]:
mixtures_and_proofs = [generate_mixture_and_proof(block) for i in range(100)]

In [663]:
puzzles = [m[0] for m in mixtures_and_proofs]

In [664]:
puzzles

['...']

In [665]:
solutions = []
for p in puzzles:
    solutions.append(solve_puzzle(p, block))

In [666]:
solutions

['3b9f74ae959965b5463be480163361bb366aecb82644683659da9e1c117686c9',
 'c22aad07f821a18ee152f48aeacb24806ed0c6c43c0d532440962276701542df',
 '3b3e3a6e1d3426878755fb9004786c4e6fd0f7972666bd4c001818423b0b26ce',
 '7853f315903ed8706855352039602d92284775cb2af4221238d10df814cddaf0',
 '0909edf3d691262387c8de3207c417912111d5a8f7ddf9fdb3a9239929051778',
 '92f19ba095a868e40d0df0d6b2a1abe430cb103f4251880d255220bc9769589c',
 '9739daba06f46a1bc284d340b37629335560d5bae1b755ab980f75036994495a',
 '92e220e9ec6d6036f8c4c5a7d1eeb2a6867c34929786c23b95c51903ceea97dc',
 '3e20fd83ca57b58d90ebdd1cecd4faaff148275dcf7230144bbc50dc9fbf5c24',
 'fa08b545eb5bb7ef82a0982aff26b15ef753584182a20af808a2852d5082a329',
 '0df601849d40f5b085aa428e4be0e6f41d381e63eaa181a0a62d595967594c14',
 '49570d1de068b03818e1e91ac5b15c2657c319104f4c5414d6f72678626c9bce',
 'a1e77a1dfcdb22d4527147d1419c03abb1a0b06219c1da7bc2d49c0693a8148f',
 '01038c4f3bc704dc848995ce065a77d86efb351d08ae3990e0a8f01397f2f4f6',
 '706d62c961cdd1c641ad76a87906dad6